# Starter API Access Tunnel
---

## Imports

In [284]:
# Dependencies
import datetime as dt
import requests
import pandas as pd
from matplotlib import pyplot
from pprint import pprint

import api_keys

In [285]:
fema_decl = pd.read_csv('DisasterDeclarationsSummaries.csv', low_memory = False)
fema_decl.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5522-CA,5522,CA,FM,2024-07-27T00:00:00.000Z,2024,Fire,BOREL FIRE,0,0,...,99029,Kern (County),24102,NaN,2024072701,9,R,2024-08-27T18:22:14.800Z,51ec819011c936f387edf10b9d5839b35419ca95,9b73e19b-d326-4992-8da1-7e658d97607c


In [286]:
# Removing unwanted/unusable columns
fema_decl_clean = fema_decl[
    [
        'disasterNumber', 'declarationRequestNumber', 'declarationTitle', 'incidentType', 
        'declarationType', 'designatedArea', 'state', 'region', 'declarationDate', 
        'incidentBeginDate', 'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared', 
        'paProgramDeclared', 'hmProgramDeclared'
    ]
].copy()
fema_decl_clean['declarationDate'] = fema_decl_clean[
    'declarationDate'
    ].str.slice_replace(10)
fema_decl_clean['incidentBeginDate'] = fema_decl_clean[
    'incidentBeginDate'
    ].str.slice_replace(10)
fema_decl_clean

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67351,9,53010,FLOOD,Flood,DR,Statewide,TX,6,1953-06-19,1953-06-19,0,0,1,1,1
67352,8,53008,FLOOD,Flood,DR,Statewide,IA,7,1953-06-11,1953-06-11,0,0,1,1,1
67353,7,53009,TORNADO,Tornado,DR,Statewide,MA,1,1953-06-11,1953-06-11,0,0,1,1,1
67354,2,53003,TORNADO & HEAVY RAINFALL,Tornado,DR,Statewide,TX,6,1953-05-15,1953-05-15,0,0,1,1,1


In [287]:
# Unique FEMA Disaster Types
disaster_types = fema_decl_clean['incidentType'].unique()
pprint(disaster_types)

array(['Fire', 'Severe Storm', 'Straight-Line Winds', 'Flood',
       'Winter Storm', 'Hurricane', 'Tornado', 'Mud/Landslide',
       'Tropical Storm', 'Snowstorm', 'Earthquake', 'Coastal Storm',
       'Other', 'Severe Ice Storm', 'Biological', 'Dam/Levee Break',
       'Typhoon', 'Volcanic Eruption', 'Freezing', 'Toxic Substances',
       'Chemical', 'Terrorist', 'Drought', 'Human Cause',
       'Fishing Losses', 'Tsunami'], dtype=object)


In [288]:
disaster_natural = pd.DataFrame(
    {
        'incidentType': [
            'Fire', 'Severe Storm', 'Straight-Line Winds', 
            'Flood', 'Winter Storm', 'Hurricane', 'Tornado',
            'Mud/Landslide', 'Tropical Storm', 'Snowstorm', 
            'Earthquake', 'Coastal Storm', 'Severe Ice Storm', 
            'Typhoon', 'Volcanic Eruption', 'Freezing', 
            'Drought', 'Tsunami'
        ]
    }
)

In [289]:
# Filter for just natural disaster types
fema_disasters = pd.merge(fema_decl_clean, disaster_natural, how = 'right', on = 'incidentType')

# TODO: Do we want renamed columns? Or do we like the current columns?
# fema_disasters = fema_disasters.rename(
#     columns = {
#         'disasterNumber': 'disaster_number', 'designatedArea': 'area',
#         'declarationRequestNumber': 'request_number', 'declarationTitle': 'title',
#         'incidentType': 'disaster_type'

#     }
# )
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [290]:
# Counts of declaration types 
# DR = Major Disasters, EM = Emergency Declaratation, FM = Fire Management)
# For more info see README (in progress)
fema_disasters['declarationType'].value_counts()

declarationType
DR    41116
EM    15991
FM     1994
Name: count, dtype: int64

In [291]:
# Counts of natural disaster types
fema_disasters['incidentType'].value_counts()

incidentType
Severe Storm           18399
Hurricane              13721
Flood                  11093
Fire                    3759
Snowstorm               3707
Severe Ice Storm        2942
Tornado                 1623
Drought                 1292
Tropical Storm          1047
Coastal Storm            637
Freezing                 301
Earthquake               228
Typhoon                  130
Winter Storm             117
Volcanic Eruption         51
Mud/Landslide             43
Tsunami                    9
Straight-Line Winds        2
Name: count, dtype: int64

In [292]:
# Validating no NaNs
fema_disasters.count()

disasterNumber              59101
declarationRequestNumber    59101
declarationTitle            59101
incidentType                59101
declarationType             59101
designatedArea              59101
state                       59101
region                      59101
declarationDate             59101
incidentBeginDate           59101
tribalRequest               59101
ihProgramDeclared           59101
iaProgramDeclared           59101
paProgramDeclared           59101
hmProgramDeclared           59101
dtype: int64

In [293]:
# Filter: Natural disasters per State
fema_disasters['state'].value_counts()

state
TX    4779
FL    2635
KY    2554
MO    2534
LA    2398
GA    2333
VA    2308
OK    2305
NC    2110
PR    1947
MS    1777
IA    1719
KS    1607
AL    1603
CA    1557
TN    1532
AR    1472
MN    1406
NE    1336
ND    1280
IN    1279
NY    1278
SD    1181
WV    1148
PA    1118
IL    1109
OH    1102
SC    1039
WA     902
WI     725
MI     595
NJ     556
OR     536
CO     531
MT     507
ME     426
MD     399
NM     380
VT     377
MA     365
NH     300
ID     276
AZ     271
CT     245
AK     211
NV     198
UT     183
RI     106
HI      97
WY      86
VI      79
MP      67
AS      66
MH      53
DE      47
FM      31
GU      20
DC      19
PW       1
Name: count, dtype: int64

In [294]:
# Filter: Natural disasters per hit area
fema_disasters['designatedArea'].value_counts()

designatedArea
Washington (County)                          510
Jefferson (County)                           462
Statewide                                    450
Jackson (County)                             450
Franklin (County)                            441
                                            ... 
Southern Ute Indian Reservation                1
Ute Mountain Indian Reservation (Also NM)      1
Nondalton (ANV/ANVSA)                          1
Salt River Pima-Maricopa Indian Community      1
Namorik (County-equivalent)                    1
Name: count, Length: 2201, dtype: int64

## Geoapify Geocoding API

In [295]:
# FEMA Disaster DataFrame with separated dates
# TODO: Possibly not needed if parsed through with `for` loop

# fema_disasters_dates = fema_disasters[['disasterNumber', 'declarationTitle', 'incidentType', 'designatedArea', 'state', 'region']]
# fema_disasters_dates['declarationYear'] = fema_disasters['declarationDate'].str.slice(0, 4)
# fema_disasters_dates['declarationMonth'] = fema_disasters['declarationDate'].str.slice(5, 7)
# fema_disasters_dates['declarationDay'] = fema_disasters['declarationDate'].str.slice(8, 10)

# fema_disasters_dates['incidentYear'] = fema_disasters['incidentBeginDate'].str.slice(0, 4)
# fema_disasters_dates['incidentMonth'] = fema_disasters['incidentBeginDate'].str.slice(5, 7)
# fema_disasters_dates['incidentDay'] = fema_disasters['incidentBeginDate'].str.slice(8, 10)
fema_disasters

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [315]:
fema_geocode_df = fema_disasters.copy()
fema_geocode_df[['area', 'areaType']] = fema_geocode_df['designatedArea'].str.split('(', n = 1, expand = True)
areas = fema_geocode_df.pop('area')
areaTypes = fema_geocode_df.pop('areaType')
areaTypes = '(' + areaTypes
fema_geocode_df.insert(5, 'area', areas)
fema_geocode_df.insert(6, 'areaType', areaTypes)
del fema_geocode_df['designatedArea']
fema_geocode_df

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,area,areaType,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe,(County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington,(County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson,(County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes,(County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern,(County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu,(County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui,(County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos,(County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry,(County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [323]:
# TODO: Make areaTypes more readable (i.e. remove parenthesis)
# TODO: Change areaType Municipio to Municipality
# TODO: Many missing value types from areaType --> Create cond. for filling missing values
    # If area name contains 'tribe', 'reservation', or 'indian' --> Fill with (Reservation)
    # If area name contains char '/' or multiple locations with regex 'and' or 'also'
    # If area type contains multiple ref or contains the county/reservation name
    # If area type contains PSMA remove that information
    # If area 


fema_geocode_df.loc[fema_geocode_df['areaType'] == '(Sisseton) Indian Reservation', :]
fema_geocode_df.loc[fema_geocode_df['areaType'] == '(Unorganized Territory of)', :]
fema_geocode_df.loc[fema_geocode_df['areaType'] == '(Township of)']
fema_geocode_df.loc[fema_geocode_df['areaType'] == '(OTSA)']

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,area,areaType,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
3868,4802,24061,SEVERE STORMS,Severe Storm,DR,Chickasaw,(OTSA),OK,6,2024-07-23,2024-03-14,1,0,0,1,1
4575,4670,22069,"SEVERE STORMS, TORNADOES, AND FLOODING",Severe Storm,DR,Creek,(OTSA),OK,6,2022-09-20,2022-05-02,1,0,0,1,1
4991,4456,19056,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",Severe Storm,DR,Creek,(OTSA),OK,6,2019-08-07,2019-05-07,1,0,0,1,1
5535,4438,19035,"SEVERE STORMS, STRAIGHT-LINE WINDS, TORNADOES,...",Severe Storm,DR,Pawnee,(OTSA),OK,6,2019-06-01,2019-05-07,0,0,0,1,1
5981,4315,17043,"SEVERE STORMS, TORNADOES, AND FLOODING",Severe Storm,DR,Pawnee,(OTSA),OK,6,2017-05-26,2017-04-28,0,0,0,1,1
10056,4690,23012,SEVERE WINTER STORM,Severe Storm,DR,Creek,(OTSA),OK,6,2023-03-03,2022-12-21,1,0,0,1,1


In [297]:
fema_geocode_df['areaType'].unique()

array(['(County)', nan, '(Parish)', '(Borough)',
       '(County)(in (P)MSA 5520,6480)', '(Census Area)',
       '(Indian Reservation)', '(County)(in PMSA 1170,3280,5440)',
       '(County)(in (P)MSA 1160,5480,8880)', '(Municipio)',
       '(County) (in (P)MSA 1120,2600,9240)', '(ANV/ANVSA)', '(Also NM)',
       '(County-equivalent)', '(County)(in PMSA 1120,4160,7090)',
       '(OTSA)', '(County)(in (P)MSA 4160,4760,6450)',
       '(County)(in PMSA 2480,6480)', '(County)(in PMSA 6060,6480)',
       '(County)(in MSA 6400,6450)', '(Also NM and UT)',
       '(Sisseton) Indian Reservation',
       '(County)(in (P)MSA 3280,5020,5480)',
       '(County)(in (P)MSA 3280,5520)',
       '(County)(in (P)MSA 1120,1200,2480,5400,6060)',
       '(County)(in PMSA 1120,1200,6060)',
       '(County)(in (P)MSA 1120,1200,5400)', '(Reservation)',
       '(County)(in (P)MSA 1120,2600,4560)',
       '(County)(in (P)MSA 4560,4760,5350)', '(District)',
       '(Island) (County-equivalent)', '(Also  AZ and UT)

In [243]:
# [print(x) for x in fema_disasters['designatedArea'].unique()]

In [162]:
geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'
geoapify_params = {
    'apiKey': api_keys.geoapify_key
    }

for index, row in fema_disasters.iterrows():
    geoapify_params['state'] = row['state']
    geocode_response = requests.get(geoapify_base, geoapify_params).json()
    pprint(geocode_response)
    break